In [10]:
import pandas as pd
import numpy as np
from scipy import stats
import math
import random

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import (
    ColumnDataSource,
    LinearColorMapper,
    LogColorMapper,
    BoxAnnotation,
)
from bokeh.transform import factor_cmap
from bokeh.palettes import Colorblind, Magma256

output_notebook()
palette = Colorblind[8]
palette_gradient = Magma256

Loading BokehJS ...

# Week 3

## Empirical rule and Z-score

In [11]:
def standardize(x):
    mean = np.mean(x)
    std = np.std(x)
    inner = lambda x: (x - mean) / std
    return inner(x)


def plot_distribution(x):
    std = np.std(x)
    mean = np.mean(x)
    median = np.median(x)

    bins = 50
    hist, edges = np.histogram(x, density=True, bins=bins)
    a, b = np.min(edges), np.max(edges)
    fill_alpha = 0.2

    p = figure(
        title=f"Distribution[{np.min(x):.2f}, {np.max(x):.2f}] len:{len(x)}, std:{std:.2f}, mean:{mean:.2f}, median:{median:.2f}"
    )
    p.vbar(x=edges[1:], top=hist, width=(b - a) / bins / 5, fill_color=palette[0])

    # Empiracal rule
    p.add_layout(
        BoxAnnotation(
            left=mean - std,
            right=mean + std,
            fill_alpha=fill_alpha,
            fill_color=palette[1],
        )
    )

    p.add_layout(
        BoxAnnotation(
            left=mean - 2 * std,
            right=mean - std,
            fill_alpha=fill_alpha,
            fill_color=palette[2],
        )
    )
    p.add_layout(
        BoxAnnotation(
            left=mean + std,
            right=mean + 2 * std,
            fill_alpha=fill_alpha,
            fill_color=palette[2],
        )
    )

    p.add_layout(
        BoxAnnotation(
            left=a, right=mean - 2 * std, fill_alpha=fill_alpha, fill_color=palette[3]
        )
    )
    p.add_layout(
        BoxAnnotation(
            left=mean + 2 * std, right=b, fill_alpha=fill_alpha, fill_color=palette[3]
        )
    )

    show(p)

In [12]:
n = 1000
mu, std = 100, 5
bins = 50
a, b = -2, 2

print("RAW")
measured = np.random.normal(mu, std, n)
plot_distribution(measured)

print("STANDARDIZED")
standardized = standardize(measured)
plot_distribution(standardized)

RAW


STANDARDIZED


## Normal approximation



In [13]:
# TODO: add code that computes z-scores, percentiles, and reverse z-score that allow to do normal approximation
# Shorthand for z-scores
x = np.random.normal(2, 0.5, 10)
z = stats.zscore(x)
print(z)

# TODO: probably add CDF and PDF as well

[ 1.22387906  0.09872527  1.87902048 -1.09364503  0.41187673 -1.18343975
  0.08560346 -1.31600634  0.42165283 -0.52766672]


## Binomial setting and Normal approximation

**Example:**

What is the probability of it most $l$ successes from $n$ attempts.
This is $l$ binomial settings that we can not sum together since they are not mutually exclusive and we would have to use *PIE* - *Principle of Inclusion and Exclusion*.

Or we can approximate it, given that the binomial curve is similar to normal curve, something like $n>50$ and use normal approximation to determine this probability.

In [14]:
# TODO create binomial setting and compare number of draws to normal distribution
n = 50
k = 4
math.comb(n, k)

230300

## Population, Parameters, Statistics, Samples and Standard Errors

* population - population with parameters e.g. mu, std
* parameter - property of population we want to estimate e.g. mean, std
* statistics - value we estimate from sample
* sample - draw from population
* standard error (SE) - SE of statistics tells us how far off statistics is from its expected value - TRUE value of parameter
    - $SE(x_n) = \sigma / \sqrt(n)$ -> **Squared root law** where $x_n$ bar is mean of sample of size $n$ from population ($\mu$, $\sigma$)
    - it is *not* affected by the size of the population itself just by the size of the sample
    - i.e. it gives us formula to get the sample size we want given the precision we want (assuming we know $\sigma$)

In [15]:
stds = [0.5, 1, 10, 20]
sample_size = np.linspace(1, 100)

assert len(stds) <= len(palette), "Can't distinguish that many standard deviations"

p = figure(y_axis_type="log", x_axis_label="sample size", y_axis_label="SE")
for idx, std in enumerate(stds):
    se = std / np.sqrt(sample_size)
    p.line(x=sample_size, y=se, line_color=palette[idx], legend_label=f"std={std}")
show(p)

## EV and SE of Sum, Percentages

**Sums**

If we are interested in sum of $n$ draws $S_n$ rather than the average $\bar{x_n}$?

$$
S_n = n * x_n
\\
E(S_n) = n * mu
\\
SE(S_n) = sqrt(n) * std
\\
$$

Careful that while with the increasing sample size $n$ the $SE(x_n)$ goes down the $SE(S_n)$ increases.


**Percentages**

*Question:* What percentage of likely voters approve of the way the US Presedent is handling his job?

Each voter either approves (labeled `1`) or not (labeled `0`). Then the answer is the sum of all the labels.

$S_n$ is the number of voters that approve.

$$
E(\text{percentage of 1s}) = \mu * 100\%
\\
SE(\text{percentage of 1s}) = \frac{\sigma}{\sqrt{n}} * 100\%
$$

These hold only for draw with replacement, however with small $\text{sample\_size} / \text{population\_size}$ it is alright.

**Expected Value and Standard Error**

Fox variable $x$ that has $k$ possible values - $x_1, x_2, ... x_k$

$$
E(\mu) = \sum[P(x_i) * x_i for x_i in values]) / n
\\
E(sum) = n * \mu
$$

And its Standard Error

$$
\sigma^2 = \sum_{i=1}^n (x_i - x_n)^2 * P(x_i)
\\
SE(sum) = \sqrt{n} * \sigma
$$


## Three types of histograms

1. Probability histogram
2. Sample histogram / Empirical histogram of real data
3. Probability histogram of statistics e.g. $S_{100} = \text{number of tails in 100 throws}$

## Law of Large numbers

> The square root law says that $SE(x_n)$, the standard error of the sample mean, goes to zero as the sample size increases.
> Therefore the $x_n$ will be close to its expected value $\mu$ if the sample size is large.

It applies to averages and therefore also percentages but not for sums as their SE increases.
This has to be sampling with replacement.

## Central Limit Theorem

> When sampling with replacement and $n$ is *large*, then the sampling distribution of the sample sum (or average or percentage) approximately follows the normal curve.

* statistics has to be sum, however averages and percentages are also sums so we can use CLT.
* thus for random variable $X$ representing some statistics we can use normal approximation - to standardize $z_i = \frac{x_i - E(X)}{SE(X)}$
* the more skewed the population distribution is the bigger sample size $n$ we need

# Week 4 - predictions, regression

## Correlation coefficient

$$
r = \frac{1}{n} * \sum_{i=1}^{n} \frac{x_i - x_n}{\sigma_x} * \frac{y_i - y_n}{\sigma{y}}
  = \frac{1}{n} * \sum_{i=1}^n zscore(x_i) * zscore(y_i)
$$

If both $x$ and $y$ are above/below their respective mean correlation coefficient is positive otherwise it is negative.

In [16]:
n = 50
x = np.linspace(0, 10, n)
y = x**2 + np.random.uniform(-20, 20, n)


def correlattion_coefficient(x, y):
    return sum(standardize(x) * standardize(y)) / n


r = correlattion_coefficient(x, y)
print(f"r = {r:.2f}")
print(f"corrcoef = {np.corrcoef(x, y)[0,1]:.2f}")

p = figure(title=f"corrcoef={r:.2f}")
p.scatter(x, y)
x = np.linspace(np.min(x), np.max(x), num=n)
p.line(x, x * r, color="red")
show(p)

r = 0.89
corrcoef = 0.89


## Regression line

Given data $(x_1, y_1), (x_2, y_2),... (x_n, y_n)$ line that goes through these points is a regression line.

Its descriptions would be $\hat{y_i} = a + b * x_i$.
* $y_i$ is the observed value
* $\hat{y_i}$ estimate of $y_i$ given by the regression line

One way to find $a$ and $b$ is to use the least square distnace method:

$$
\sum_{i=1}^n (y_i - \hat{y_i})^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2
$$